In [103]:
# %config IPCompleter.greedy=True
# import libraries
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
# import requests
# from lxml import html as lh
# import traceback
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
# from timeit import default_timer as timer

### Settings for browser

In [2]:
# create options object for chromium webdriver
options = webdriver.ChromeOptions()
# setup path for brave binary (executable file)
options.binary_location = "C:\\Program Files (x86)\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"
# add extensions for brave to open new instance of browser with VPN extension activated
# for this purpose I used extensions source code extractor on http://crxextractor.com/
# using link of extension found in store
options.add_extension(r"extension_4_1_0_0.crx")
# start chrome maximized
options.add_argument("start-maximized")
# downloaded chromium driver binaries and place it in project directory
chrome_driver_binary = "chromedriver.exe"

### Chrome Driver

In [141]:
# Chrome - controls browser
driver = webdriver.Chrome(chrome_driver_binary, options=options)
# choose which site you want to visit
driver.get("https://www.otodom.pl/sprzedaz/mieszkanie/wroclaw/?search%5Bfilter_float_price_per_m%3Afrom%5D=1&search%5Bfilter_float_price%3Afrom%5D=1&search%5Border%5D=quality_score&search%5Bcity_id%5D=39&nrAdsPerPage=24")
# check if it open right site by site title
assert "Mieszkanie na sprzedaż w Wrocław, dolnośląskie - www.otodom.pl" in driver.title

#### Get page content and parse it with beautifulSoup

In [142]:
# attributes to collect
offer_id = []
offer_district = []
offer_price = []
offer_price_per_m2 = []
offer_room_number = []
offer_flat_surface_m2 = []

offer_floor = []
offer_floor_number = []
offer_prod_year = []
offer_rent = []
offer_market = []
offer_building_type = []
offer_finish_condition = []


In [143]:
# helpers
current_page = 0
page_number = 0
doubled_offers = 0
dont_get_building_type = 0
offer_row = {}
offer_overview_posibilities = ["Piętro:", "Liczba pięter:", "Rok budowy:", "Czynsz:", "Rynek:", "Rodzaj zabudowy:", 
                               "Stan wykończenia:"]

In [144]:
while current_page <= page_number:
    # get full page source
    content = driver.page_source
    
    # parse it with beautiflesoup
    soup = bs(content)
    
    #iterate through all articles (offer details are in articles)
    for offer_article in soup.find_all("article"):
        
        # if article not in history then...
        if offer_article["id"] not in offer_id:   

            # add offer id to history
            offer_id.append(offer_article["id"])
            
            # add offer price
            price = offer_article.find('li', attrs={'class':'offer-item-price'}).get_text()
            offer_price.append(price.strip())
            
            # add offer room number
            room_number = offer_article.find('li', attrs={'class':'offer-item-rooms hidden-xs'}).get_text()
            offer_room_number.append(room_number)
            
            # add offer surface 
            flat_m2 = offer_article.find('li', attrs={'class':'hidden-xs offer-item-area'}).get_text()
            offer_flat_surface_m2.append(flat_m2)
            
            # add offer price per m2
            price_per_m2 = offer_article.find('li', attrs={'class':'hidden-xs offer-item-price-per-m'}).get_text()
            offer_price_per_m2.append(price_per_m2)
            
            # add offer district
            district = offer_article.find('p', attrs={"class":"text-nowrap"}).get_text()
            offer_district.append(district)
            
            # open new tab process
            # store current tab for return
            main_window = driver.current_window_handle
            
            # get URL of 
            offer_details_link = soup.find(attrs={"id":offer_article["id"]}).get("data-url")

            # open new tab and switch driver to it
            driver.execute_script("window.open();")
            driver.switch_to.window(driver.window_handles[1])
            
            # set driver to get new offer details site
            driver.get(offer_details_link)
            
            # collect data from offer details site           
            # flat offer overview contains rest of attributes
            WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, "css-dwmx8v-Fe")))
            
            # save overview element to variable
            overview = driver.find_element_by_class_name("css-dwmx8v-Fe")
            details = overview.find_elements_by_tag_name("li")
            
            # add all li text in list
            overview_categories = [category.text for category in details]
            
            # search for data in list by substring             
            floor = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[0] in s]
            floor_number = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[1] in s]
            production_year = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[2] in s]
            rent = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[3] in s]
            market = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[4] in s]
            building_type = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[5] in s]
            finish_condition = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[6] in s]
            
            if floor:
                offer_floor.append(floor[0])
            else:
                offer_floor.append(None)
            if floor_number:
                offer_floor_number.append(floor_number[0])
            else:
                offer_floor_number.append(None)
            if production_year:
                offer_prod_year.append(production_year[0])
            else:
                offer_prod_year.append(None)
            if rent:
                offer_rent.append(rent[0])
            else:
                offer_rent.append(None)
            if market:
                offer_market.append(market[0])
            else:
                offer_market.append(None)
            if building_type:
                offer_building_type.append(building_type[0])
            else:
                offer_building_type.append(None)
            if finish_condition:
                offer_finish_condition.append(finish_condition[0])
            else:
                offer_finish_condition.append(None)
                
            # close tab
            driver.close()

            # back to the main window
            driver.switch_to.window(main_window)
        else:
            doubled_offers += 1
            print(f"Number of doubled offers: {doubled_offers}")
            
    current_page += 1

KeyboardInterrupt: 

In [153]:
# test = ['Powierzchnia: 82 m²', 'Liczba pokoi: 4', 'Rynek: wtórny', 'Rodzaj zabudowy: apartamentowiec', 
#         'Piętro: parter', 'Liczba pięter: 3', 'Materiał budynku: keramzyt', 'Okna: drewniane', 'Ogrzewanie: kotłownia', 
#         'Rok budowy: 2015', 'Stan wykończenia: do zamieszkania', 'Czynsz: 680 zł', 'Forma własności: pełna własność']

# indices = [s for i, s in enumerate(test) if offer_overview_posibilities[6] in s]
# indices
# x = driver.find_element_by_partial_link_text(offer_overview_posibilities[1])
# x
# print((offer_id))
# print((offer_district))
# print((offer_price))
# print((offer_price_per_m2))
# print((offer_floor))
# print((offer_floor_number))
# print((offer_prod_year))
# print((offer_rent))
# print((offer_market))
# print((offer_building_type))
# print((offer_finish_condition))
# print((offer_flat_surface_m2))
# print((offer_room_number))
offer_price.pop()

'509 000 zł'

In [154]:
print("id: ",len(offer_id))
print("distric: ",len(offer_district))
print("price: ",len(offer_price))
print("price per m2: ",len(offer_price_per_m2))
print("floor: ",len(offer_floor))
print("floor number: ",len(offer_floor_number))
print("prod year: ",len(offer_prod_year))
print("rent: ",len(offer_rent))
print("market: ",len(offer_market))
print("build type: ",len(offer_building_type))
print("finish cond: ",len(offer_finish_condition))
print("flat surface: ",len(offer_flat_surface_m2))
print("room number: ",len(offer_room_number))
offer_row = {"id":offer_id,"district":offer_district,"price":offer_price,"price_per_m2":offer_price_per_m2,
             "floor":offer_floor,"floor_number":offer_floor_number,"prod_year":offer_prod_year,"rent":offer_rent,
            "market":offer_market,"building_type":offer_building_type,"finish_condition":offer_finish_condition,
            "flat_surface_m2":offer_flat_surface_m2,"room_number":offer_room_number}

df = pd.DataFrame(offer_row)
df.head()


id:  7
distric:  7
price:  7
price per m2:  7
floor:  7
floor number:  7
prod year:  7
rent:  7
market:  7
build type:  7
finish cond:  7
flat surface:  7
room number:  7


,id,district,price,price_per_m2,floor,floor_number,prod_year,rent,market,building_type,finish_condition,flat_surface_m2,room_number
0,offer-item-ad_id48IAh,"Mieszkanie na sprzedaż: Wrocław, Krzyki",585 000 zł,9 286 zł/m²,Piętro: 3,Liczba pięter: 3,Rok budowy: 2008,Czynsz: 375 zł,Rynek: wtórny,Rodzaj zabudowy: apartamentowiec,Stan wykończenia: do zamieszkania,63 m²,2 pokoje
1,offer-item-ad_id47fmf,"Mieszkanie na sprzedaż: Wrocław, Krzyki, Południe",289 000 zł,14 450 zł/m²,Piętro: 1,Liczba pięter: 4,None,None,Rynek: wtórny,Rodzaj zabudowy: kamienica,Stan wykończenia: do zamieszkania,20 m²,1 pokój
2,offer-item-ad_id46G51,"Mieszkanie na sprzedaż: Wrocław, Fabryczna, Gr...",329 000 zł,8 164 zł/m²,Piętro: parter,Liczba pięter: 4,Rok budowy: 1950,Czynsz: 340 zł,Rynek: wtórny,Rodzaj zabudowy: kamienica,Stan wykończenia: do zamieszkania,"40,30 m²",2 pokoje
3,offer-item-ad_id48SH6,"Mieszkanie na sprzedaż: Wrocław, Krzyki, Jagodno",495 000 zł,8 763 zł/m²,Piętro: 4,Liczba pięter: 4,Rok budowy: 2019,Czynsz: 360 zł,Rynek: pierwotny,Rodzaj zabudowy: blok,Stan wykończenia: do zamieszkania,"56,49 m²",3 pokoje
4,offer-item-ad_id48SRy,"Mieszkanie na sprzedaż: Wrocław, Fabryczna, Mu...",489 000 zł,7 641 zł/m²,Piętro: 3,Liczba pięter: 4,Rok budowy: 2002,Czynsz: 400 zł,Rynek: wtórny,Rodzaj zabudowy: blok,None,64 m²,4 pokoje


In [ ]:
offer_districts_examples = {"Stare Miasto":["Szczepin", 
                                            "Stare Miasto", 
                                            "Przedmieście Świdnickie"], 
                            "Psie Pole":["Karłowice - Różanka", 
                                         "Osobowice - Rędzin", 
                                         "Lipa Piotrowska", 
                                         "Świniary", 
                                         "Widawa", 
                                         "Polanowice - Poświętne - Ligota",
                                         "Sołtysowice", 
                                         "Kowale", 
                                         "Strachocin - Swojczyce - Wojnów", 
                                         "Psie Pole - Zawidawie", 
                                         "Pawłowice"], 
                            "Krzyki":["Powstańców Śląskich", 
                                      "Huby", 
                                      "Tarnogaj", 
                                      "Przedmieście Oławskie", 
                                      "Księże", 
                                      "Brochów",
                                      "Bieńkowice", 
                                      "Gaj", 
                                      "Borek", 
                                      "Klecina", 
                                      "Krzyki - Partynice", 
                                      "Jagodno", 
                                      "Wojszyce", 
                                      "Ołtaszyn"], 
                            "Fabryczna":["Pracze Odrzańskie", 
                                         "Leśnica", 
                                         "Jerzmanowo - Jarnołtów - Strachowice - Osiniec", 
                                         "Maślice", 
                                         "Żerniki", 
                                         "Pilczyce - Kozanów - Popowice Płn.", 
                                         "Gądów - Popowice Płd.", 
                                         "Kuźniki", 
                                         "Nowy Dwór", 
                                         "Muchobór Wielki", 
                                         "Muchobór Mały", 
                                         "Gajowice", 
                                         "Grabiszyn - Grabiszynek", 
                                         "Oporów"], 
                            "Śródmieście":["Kleczków", 
                                           "Nadodrze", 
                                           "Ołbin", 
                                           "Plac Grunwaldzki", 
                                           "Zacisze - Zalesie - Szczytniki", 
                                           "Biskupin - Sępolno - Dąbie - Bartoszowice"]}


##### Raw data is in html format, need to get raw text from tag then strip text and remove "zł" from data

In [ ]:
start = timer()

offer_price = []
offer_id = set()
offer_room_number = []
offer_flat_sqm = []
offer_price_for_sqm = []
offer_floor = []
offer_floor_number = []
offer_prod_year = []
offer_rent = []
offer_market = []
offer_building_type = []
offer_finish_condition = []
offer_district = []

offer_id_link = {}
offer_overview_posibilities = ["Piętro:", "Liczba pięter:", "Rok budowy:", "Czynsz:", "Rynek:", "Rodzaj zabudowy:", 
                               "Stan wykończenia:"]
current_page = 0
page_number = 10

while current_page <= page_number:
    content = driver.page_source
    soup = bs(content)
#     driver.implicitly_wait(15)
    for article in soup.find_all("article"):
        article_list = []
        if article['id'] not in article_list:
            article_id = article['id']
            article_list.append(article_id)
            price = article.find('li', attrs={'class':'offer-item-price'})
            room_number = article.find('li', attrs={'class':'offer-item-rooms hidden-xs'})
            flat_sqm = article.find('li', attrs={'class':'hidden-xs offer-item-area'})
            price_for_sqm = article.find('li', attrs={'class':'hidden-xs offer-item-price-per-m'})

            offer_id.add(article_id)
            offer_price.append(price.get_text())
            offer_room_number.append(room_number.get_text())
            offer_flat_sqm.append(flat_sqm.get_text())
            offer_price_for_sqm.append(price_for_sqm.get_text())

            # open ofert in new tag
            main_window = driver.current_window_handle
            # get url of offer
            get_article_tag = driver.find_element_by_id(article_id)
            offer_url = get_article_tag.get_attribute("data-url")

            offer_id_link[article_id] = offer_url

#             # open new tab and switch driver to it
#             driver.execute_script("window.open();")
#             driver.switch_to.window(driver.window_handles[1])
#             # get site
#             driver.get(offer_url)
#             print('dada')
#             driver.implicitly_wait(15)
#             # get district names
#             try:
#                 distric_find = driver.find_element_by_class_name("css-12hd9gg")
#                 offer_district.append(distric_find.text)
#             except:
#                 print("Couldn't find district")
#             driver.implicitly_wait(25)
# #             try:
#             offer_overview = driver.find_element_by_class_name("section-overview")
#             offer_details = offer_overview.find_elements_by_tag_name("li")
# #             print("Found")
#             for elem in offer_details:
#                 text = elem.text
# #                         print(text)
#                 if offer_overview_posibilities[0] in text:
#                     offer_floor.append(text)
#                 if offer_overview_posibilities[1] in text:
#                     offer_floor_number.append(text)
#                 if offer_overview_posibilities[2] in text:
#                     offer_prod_year.append(text)
#                 if offer_overview_posibilities[3] in text:
#                     offer_rent.append(text)
#                 if offer_overview_posibilities[4] in text:
#                     offer_market.append(text)
#                 if offer_overview_posibilities[5] in text:
#                     offer_building_type.append(text)
#                 if offer_overview_posibilities[6] in text:
#                     offer_finish_condition.append(text)
# #             except:
# #                 print("Coudn't find")

#             driver.close()

#             # back to the main window
#             driver.switch_to.window(main_window)
        else:
            print('Oferta powtórzona')
    
#     driver.implicitly_wait(15)

    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'pager-next'))).click()
    current_page += 1

end = timer()
print(end - start)

In [ ]:
# offer_price.append(price.get_text().replace("zł",'').replace(" ","").strip())
# cena = int(offer_price[1])
# cena2 = int(offer_price[2])
# cena+cena2
# print(offer_id)
# print((offer_price))
# print(offer_room_number)
# print(offer_flat_sqm)
# print(offer_price_for_sqm)
# len(offer_price)
# len(offer_id)
# counter
# print(offer_id_link)
# print(len(offer_district))
# print((offer_building_type))
# print(len(offer_floor_number))

In [ ]:
# for i in range(3):
#     WebDriverWait(driver,10).until(EC.presence_of_element_located(By.CLASS_NAME)).click()
#     pager = driver.find_element_by_class_name('pager-next')
#     a_tag_next_page = pager.find_element_by_tag_name('a')
#     a_tag_next_page.click()
#     print(a_tag_next_page)
#     WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//form[@id='pagerForm']/ul/li[@class='pager-next']/a"))).click()
    

## Scrap table data from gp-sms.pl
### Based on article from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [ ]:
options = webdriver.ChromeOptions()
options.binary_location = "C:/Program Files (x86)/BraveSoftware/Brave-Browser/Application/brave.exe"
chrome_driver_binary = "chromedriver.exe"
driver = webdriver.Chrome(chrome_driver_binary, options=options)
server_name = []
gold_amount = []

# elem = driver.find_element_by_class_name("table-gold-available")
driver.get("http://gp-sms.pl/")
content = driver.page_source
soup = bs(content)
doc = lh.fromstring(content)
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    name=t.text_content()
    col.append((name,[]))

#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i > 0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [ ]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [ ]:
df.to_excel('gold_amount_gp_sms_scrap2.xlsx', index=False, encoding='utf-8')

In [ ]:
df.to_csv('gold_amount_gp_sms_scrap.csv', index=False, encoding='utf-8')

In [ ]:
df = pd.read_excel("gold_amount_gp_sms_scrap.xlsx", dtype={'Ilość CC': float})
df.to_excel('gold_amount_gp_sms_scrap2.xlsx', index=False, encoding='utf-8')

In [ ]:
df.head(100)

## Tutorial from selenium docs on custom example

In [ ]:
driver.get("http://gp-sms.pl/")
assert "Tibia GP za SMS" in driver.title
elem = driver.find_element_by_id("searchbar-gold-available")
elem.clear()
elem.send_keys("Harmonia")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source

## Tutorial from https://selenium-python.readthedocs.io/

In [ ]:
options = webdriver.ChromeOptions()
options.binary_location = "C:/Program Files (x86)/BraveSoftware/Brave-Browser/Application/brave.exe"
chrome_driver_binary = "chromedriver.exe"
driver = webdriver.Chrome(chrome_driver_binary, options=options)
driver.get("http://www.python.org")
assert "Python" in driver.title
elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("dsafdaga")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
# driver.close()